In [26]:
import os, operator, nltk, csv
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import networkx as nx

In [47]:
dataset_dir = 'Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/'
    
for subdir, dirs, files in os.walk(dataset_dir):
    # iterate over all files in train/test directory
    for f in files:
        if not f.endswith('.txt') or f.endswith('justTitle.txt'):
            continue
        fopen = open(subdir+'/'+f, 'r')#, encoding="utf-8")
        text = fopen.read().decode('utf-8')
        # tokenizing the text and filtering nouns / verbs etc
        tokens = nltk.word_tokenize(text)
        tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

        def filter_words(key):
            return True if key.startswith('NN') or key.startswith('J') or key.startswith('VB') else False
        filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))
        
        # creating local maps for nodes and edges
        dict_text = defaultdict(int)
        for t in filt_text:
            dict_text[t] += 1
        edges, nodes, nodes_number = [], {}, {}
        c = 0
        for d in dict_text:
            if d in nodes:
                continue
            nodes[d] = c
            nodes_number[c] = d
            c += 1
            
        # create edges for graph-of-words
        k = min(10, len(filt_text)-1) ## window size
        for i in range(len(filt_text)-k):
            for j in range(i+1, i+k+1):
                wt = j-i  ## this is the weight of the edge (distance between words)
                edges.append((nodes[filt_text[i]], nodes[filt_text[j]], wt))

        newDir = '/'.join(subdir.split('/')[:-2])
        newFile = newDir + '/graphEdges/' + subdir.split('/')[-1] + '/' + f
        newMapFile = newFile[:-4]+'-map.txt'
        with open(newFile, "w+") as wrt:
            out = csv.writer(wrt)
            out.writerow(('node1', 'node2', 'weight'))
            for e in edges:
                out.writerow(e)
        with open(newMapFile, "w+") as wrt:
            out = csv.writer(wrt)
            out.writerow(('word', 'node'))
            for n in nodes:
                out.writerow((n.encode('utf-8'), nodes[n]))

In [53]:
dataset_graph_dir = 'Dataset/500N-KPCrowd-v1.1/graphEdges/'
    
for subdir, dirs, files in os.walk(dataset_graph_dir):
    # iterate over all files in train/test directory
    for f in files:
        if f.endswith('-map.txt'):
            continue
        
        edges = []
        nodeToWordMap, wordToNodeMap = {}, {}
       
        # reading map of nodes to words
        with open(subdir + '/' + f[:-4] + '-map.txt', "r") as fopen:
            reader = csv.reader(fopen)
            graph_nodes_map  = list(reader)[1:]
        for g in graph_nodes_map:
            nodeToWordMap[int(g[1])] = g[0]
            wordToNodeMap[g[0]] = int(g[1])

        # reading edge list for the graph
        with open(subdir + '/' + f, "r") as fopen:
            reader = csv.reader(fopen)
            graph_edges  = list(reader)[1:]
        for g in graph_edges:
            edges.append((int(g[0]), int(g[1]), int(g[2])))

In [183]:
f = open('Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/train/art_and_culture-20893614.txt', 'r')
text = f.read()

In [184]:
# tokenizing the text and filtering nouns / verbs etc
tokens = nltk.word_tokenize(text)
tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

def filter_words(key):
    return True if key.startswith('NN') or key.startswith('J') or key.startswith('VB') else False
filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))

In [185]:
# creating local maps for nodes and edges
dict_text = defaultdict(int)
for t in filt_text:
    dict_text[t] += 1
edges = []
nodes = {}
nodes_number = {}
c = 0
for d in dict_text:
    if d in nodes:
        continue
    nodes[d] = c
    nodes_number[c] = d
    c += 1

In [186]:
# create edges for graph-of-words
k = 5 ## window size
for i in range(len(filt_text)-k):
    for j in range(i+1, i+k+1):
        wt = j-i  ## this is the weight of the edge (distance between words)
        edges.append((nodes[filt_text[i]], nodes[filt_text[j]], wt))

In [187]:
# create a file and store the edges
#fw = open('Dataset/500N-KPCrowd-Edges/train/art_and_culture-20893614.txt', 'w')
#fw.write(edges)

In [188]:
# create graph-of-words
G=nx.Graph()

### TODO: read from file to add edges
G.add_weighted_edges_from(edges)

In [189]:
# extracting keywords using simple degree centrality
centrality = nx.degree_centrality(G)
sorted_centr = sorted(centrality, key=centrality.get, reverse=True)
for i in sorted_centr[:30]:
    print nodes_number[i]

Lopilato
Buble
Aires
Buenos
Argentine
fans
Michael
pop
ceremony
civil
actress
month
Luisana
Thursday
wedding
concert
star
TV
Argentina
wore
Canadian
AP
including
mansion
top-grossing
March
rose
young
bouquet
rice


In [190]:
# extracting keywords using page-rank
page_ranks = nx.pagerank(G)
sorted_page_ranks = sorted(page_ranks, key=page_ranks.get, reverse=True)
for i in sorted_page_ranks[:30]:
    print nodes_number[i]

Lopilato
Buble
Buenos
Aires
fans
Argentine
pop
Michael
ceremony
civil
actress
month
Luisana
concert
star
Thursday
wedding
TV
wore
day
Canadian
messages
Argentina
AP
sent
Twitter
Facebook
followers
Juan
Little
